In [1]:
# Load packages
import os
import pandas as pd 
import geopandas as gpd 
from shapely.geometry import Point
import numpy as np 

In [6]:
# Load osm_building Features and calculate polygon centroids
home_path = os.getenv('HOME')
osm_building_shp = home_path+'/GeoData/OSM/algeria-latest-free.shp/gis_osm_buildings_a_free_1.shp'
osm_building_gdf = gpd.read_file(osm_building_shp)
osm_building_gdf.head()

,osm_id,code,fclass,name,type,geometry
0,23453590,1500,building,None,None,"POLYGON ((3.28630 36.72337, 3.28680 36.72384, ..."
1,23453621,1500,building,None,industrial,"POLYGON ((3.28839 36.72555, 3.28945 36.72653, ..."
2,23453649,1500,building,None,industrial,"POLYGON ((3.29652 36.73153, 3.29695 36.73203, ..."
3,26224008,1500,building,Hilton Alger,hotel,"POLYGON ((3.15595 36.73836, 3.15634 36.73856, ..."
4,26224288,1500,building,Crystal Lounge,None,"POLYGON ((3.15561 36.73811, 3.15583 36.73823, ..."


In [7]:
# Set the correct CRS and reproject
osm_building_gdf = osm_building_gdf.set_crs("EPSG:4326")
osm_building_gdf = osm_building_gdf.to_crs("EPSG:3857")

osm_building_gdf.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World - 85°S to 85°N
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
# Load the targeted dataset
pois_path = '../dz-datasets/notaries/'
pois = gpd.read_file(pois_path + 'pois_checked_in_out_wilayas.geojson')
pois.head()

,num,name,address,lat,lon,code_wil,to_check,geometry
0,58,AROUCHE HOCINE,CITE BEN HAMMADI ALLAL Timimoun,29.261691,0.241596,1,False,POINT (0.24160 29.26169)
1,55,BENHADJ AHMED,CITE DES 40 LOGTS LOCAL 25 Adrar,28.017440,-0.264250,1,False,POINT (-0.26425 28.01744)
2,56,BARIK BOUDJEMAA,CITE 103 LOGTS BP 575 Adrar,28.017440,-0.264250,1,False,POINT (-0.26425 28.01744)
3,17,BOUDJELTIA ABDELKADER,42 RUE DES MARTYRS Chlef,36.157966,1.337282,2,False,POINT (1.33728 36.15797)
4,202,BEN DAOUDIA DRISS,1 CITE 136 LOGTS BT I Chlef,36.157966,1.337282,2,False,POINT (1.33728 36.15797)


In [9]:
# Define source CRS to geographic and project to a Pseudo-Mercator
# Useful for spatial analysis (buffer, intersect, ...)

pois = pois.set_crs("EPSG:4326")
pois = pois.to_crs("EPSG:3857")
pois.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World - 85°S to 85°N
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
# Replace DataFrame geometries with buffers and write it to a file
pois['buffer'] = pois.geometry.buffer(500)
buffers = pois[['num', 'buffer']]
buffers = buffers.rename(columns={'buffer': 'geometry'})
buffers = gpd.GeoDataFrame(buffers)
buffers = buffers.set_crs("EPSG:3857")
#gpd.GeoDataFrame(buffers).to_file(pois_path + "buffers.geojson", driver='GeoJSON')
buffers.head()

,num,geometry
0,58,"POLYGON ((27394.388 3408995.798, 27391.981 340..."
1,55,"POLYGON ((-28916.142 3251172.790, -28918.550 3..."
2,56,"POLYGON ((-28916.142 3251172.790, -28918.550 3..."
3,17,"POLYGON ((149365.585 4322379.131, 149363.177 4..."
4,202,"POLYGON ((149365.585 4322379.131, 149363.177 4..."


In [11]:
# Search for buildings that intersect the locations buffers
res_intersection = gpd.overlay(buffers, osm_building_gdf, how='intersection')
res_intersection.count()

num         78146
osm_id      78146
code        78146
fclass      78146
name         6801
type        18732
geometry    78146
dtype: int64

In [12]:
res_intersection.head()

,num,osm_id,code,fclass,name,type,geometry
0,58,745144709,1500,building,None,None,"POLYGON ((26441.936 3409207.201, 26453.428 340..."
1,17,427245931,1500,building,None,residential,"POLYGON ((149114.689 4322812.057, 149123.703 4..."
2,202,427245931,1500,building,None,residential,"POLYGON ((149114.689 4322812.057, 149123.703 4..."
3,121,427245931,1500,building,None,residential,"POLYGON ((149114.689 4322812.057, 149123.703 4..."
4,122,427245931,1500,building,None,residential,"POLYGON ((149114.689 4322812.057, 149123.703 4..."


In [13]:
# Write intersected building into a GeoJSON file
res = res_intersection[['num', 'geometry']]
#res.to_file(pois_path + "intersection.geojson", driver='GeoJSON')
res.head()

,num,geometry
0,58,"POLYGON ((26441.936 3409207.201, 26453.428 340..."
1,17,"POLYGON ((149114.689 4322812.057, 149123.703 4..."
2,202,"POLYGON ((149114.689 4322812.057, 149123.703 4..."
3,121,"POLYGON ((149114.689 4322812.057, 149123.703 4..."
4,122,"POLYGON ((149114.689 4322812.057, 149123.703 4..."


In [14]:
# Group results of intersection and add a to_check column (if intersect then doesn't need to check)
res_grouped = res [['num']]
res_grouped = res_grouped.groupby('num').count()
res_grouped['to_check_osm'] = False
res_grouped.count()

to_check_osm    624
dtype: int64

In [15]:
# Merge with DataFrame and set geometry to centroids
pois_gdf = pois.merge(res_grouped, left_on='num', right_on='num', how='left')
pois_gdf.drop(columns=['buffer'], inplace=True)
pois_gdf.count()

num             665
name            665
address         665
lat             665
lon             665
code_wil        665
to_check        665
geometry        665
to_check_osm    624
dtype: int64

In [16]:
# Update to_check column :if doesn't intersect then we need to check it manually
pois_gdf['to_check_osm'] = pois_gdf.apply(lambda row: row.to_check_osm != False, axis=1)
pois_gdf[pois_gdf['to_check_osm'] != False]

,num,name,address,lat,lon,code_wil,to_check,geometry,to_check_osm
1,55,BENHADJ AHMED,CITE DES 40 LOGTS LOCAL 25 Adrar,28.017440,-0.264250,1,False,POINT (-29416.142 3251172.790),True
2,56,BARIK BOUDJEMAA,CITE 103 LOGTS BP 575 Adrar,28.017440,-0.264250,1,False,POINT (-29416.142 3251172.790),True
30,259,DELINDA KAMEL,15 RUE CHENAF AMAR QUARTIER VERDURE Batna,35.532190,6.168609,5,False,POINT (686686.457 4236440.873),True
40,19,BENHOUDA BRAHIM,5 BD BENFLIS Batna,35.532190,6.168609,5,False,POINT (686686.457 4236440.873),True
105,219,ABDELAOUI ZOULIKHA,CITE DES 120 LOGTS BT 1 APPT 3 Bouira,36.538950,3.675159,10,False,POINT (409116.828 4375035.759),True
126,318,REBII RAFIK,43 CITE 84 LOGTS Bir El Ater,34.749839,8.057753,12,False,POINT (896984.916 4129937.016),True
130,648,BALOUT TOUHAMI,116 CITE KASMI ABDELKADER RTE BELABES Ouled Mi...,34.904412,-1.034236,13,False,POINT (-115130.603 4150898.638),True
162,146,TOUMI AKLI,38 AVENUE KHOUDJA KHALED Tizi Ouzou,36.735307,3.984973,15,False,POINT (443605.132 4402275.909),True
169,156,BELACEL BOUBEKEUR,RUE AMIROUCHE AIT BEN AMARA Tizi Ouzou,36.738997,4.010825,15,False,POINT (446483.008 4402788.551),True
189,616,BOULEKRIAT BOUDJEMAA,CITE 47 LOGTS BT 05 02 Kheraicia,36.667911,2.998277,16,False,POINT (333766.647 4392918.357),True


In [17]:
final_gdf = pois_gdf
final_gdf['to_check'] = (pois_gdf['to_check'] == True) | (pois_gdf['to_check_osm'] == True)   
final_gdf[final_gdf.to_check == True]

,num,name,address,lat,lon,code_wil,to_check,geometry,to_check_osm
1,55,BENHADJ AHMED,CITE DES 40 LOGTS LOCAL 25 Adrar,28.017440,-0.264250,1,True,POINT (-29416.142 3251172.790),True
2,56,BARIK BOUDJEMAA,CITE 103 LOGTS BP 575 Adrar,28.017440,-0.264250,1,True,POINT (-29416.142 3251172.790),True
30,259,DELINDA KAMEL,15 RUE CHENAF AMAR QUARTIER VERDURE Batna,35.532190,6.168609,5,True,POINT (686686.457 4236440.873),True
40,19,BENHOUDA BRAHIM,5 BD BENFLIS Batna,35.532190,6.168609,5,True,POINT (686686.457 4236440.873),True
92,189,DRAFLI HOURIA,RUE MOHAMED BOURAS Boufarik,36.663154,3.106977,9,True,POINT (345867.086 4392258.241),False
105,219,ABDELAOUI ZOULIKHA,CITE DES 120 LOGTS BT 1 APPT 3 Bouira,36.538950,3.675159,10,True,POINT (409116.828 4375035.759),True
126,318,REBII RAFIK,43 CITE 84 LOGTS Bir El Ater,34.749839,8.057753,12,True,POINT (896984.916 4129937.016),True
130,648,BALOUT TOUHAMI,116 CITE KASMI ABDELKADER RTE BELABES Ouled Mi...,34.904412,-1.034236,13,True,POINT (-115130.603 4150898.638),True
162,146,TOUMI AKLI,38 AVENUE KHOUDJA KHALED Tizi Ouzou,36.735307,3.984973,15,True,POINT (443605.132 4402275.909),True
169,156,BELACEL BOUBEKEUR,RUE AMIROUCHE AIT BEN AMARA Tizi Ouzou,36.738997,4.010825,15,True,POINT (446483.008 4402788.551),True


In [18]:
# Drop unecessary columns
final_gdf = final_gdf.drop(columns=['to_check_osm'], axis=1)

In [19]:
# Reproject to geograĥic and Write to a GeoPackage file
final_gdf = final_gdf.set_crs("EPSG:3857")
final_gdf = final_gdf.to_crs("EPSG:4326")
final_gdf.to_file(pois_path + "pois_to_check.geojson", driver='GeoJSON')